In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd
import time
import os
import re

def is_target_file(file_name):
    today = datetime.today().strftime('%Y-%m-%d')
    return re.search("data-" + today + ".*\.xls$", file_name) is not None
    

def MultiDownloads(str,folder_path="/Users/sinianzhang/Downloads"):
    
    # browser driver
    driver = webdriver.Chrome()

    # url of website
    url = "https://dev.parse-health.org/shiny/ARCH/"
    driver.get(url)
    
    # sidebar button
    button = driver.find_element(By.CLASS_NAME, 'sidebar-toggle')
    button.click()
    time.sleep(60)
    # keyword input box
    input_box = driver.find_element(By.ID, 'side-searchbox2_text')
    input_box.clear()
    input_box.send_keys(str)
    # Search Button
    button = driver.find_element(By.ID, 'side-searchbox2_search')
    button.click()
    time.sleep(30)

    # Get all of the elements
    table_element = driver.find_element(By.ID, 'DataTables_Table_1')
    rows_elements = table_element.find_elements(By.TAG_NAME, 'tr')

    DownLoadTimes = len(rows_elements)

    for d in range(1,DownLoadTimes):
        
        # click the d row
        rows_elements[d].click()
        time.sleep(10)
        
        # submit
        button = driver.find_element(By.ID,'side-gobutton')
        button.click()
        time.sleep(10)
        
        # table of nodes
        tab = driver.find_element(By.CSS_SELECTOR, 'a[data-value="Table of connected nodes"]')
        tab.click()
        time.sleep(10)
        
        # download the excel
        button = driver.find_element(By.ID,'btn-downloadData')
        button.click()
        
        # refresh the website
        driver.refresh()
        time.sleep(10)  
        
        # search again
        button = driver.find_element(By.CLASS_NAME, 'sidebar-toggle')
        button.click()
        time.sleep(10)
        input_box = driver.find_element(By.ID, 'side-searchbox2_text')
        input_box.clear()
        input_box.send_keys(str)
        button = driver.find_element(By.ID, 'side-searchbox2_search')
        button.click()
        time.sleep(10)  
        
        table_element = driver.find_element(By.ID, 'DataTables_Table_1')
        rows_elements = table_element.find_elements(By.TAG_NAME, 'tr')
        
    driver.quit()
    
    ## data processing
    target_files = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if is_target_file(filename):
                target_files.append(os.path.join(dirpath, filename))
                
    ## merge the data
    tempdataframes = []

    for file in target_files:
        
        ## add the data
        data = pd.read_csv(file, sep='\t')
        tempdataframes.append(data)
        os.remove(file)


    ## merge the data 
    finaldataframe = pd.concat(tempdataframes, ignore_index=True)
    
    ## add the disease name
    finaldataframe['disease'] = str   
        
    # save the datas
    finaldataframe.to_csv(folder_path +'/' + str + ".csv",encoding='utf_8_sig',index=False)


In [2]:
for term in ["renal cell carcinoma","cancer recurrence"]:
    MultiDownloads(term)